# Notebook Title

**Authorship:**
Author, *MM/DD/YYYY*
***
**Description:**
Notebook to do some cool stuff
***
**TODOs:**
 - <font color='green'> Done TODO </font>
 - <font color='orange'> WIP TODO </font>
 - <font color='red'> Queued TODO </font>
***

## Set-up

In [2]:
# The classics

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [3]:
# Directory definition
dir = "/path/to/dir"

## Part 1
Description

## Part 2
Description

# Scratch
Place for old or testing code

# References